In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
import torch
import numpy as np

In [ ]:
api_key = ""
url = "https://api.deepseek.com/v1/chat/completions"
headers = {"Authorization": f"Bearer {api_key}"}

In [ ]:
C_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model_name = "Qwen/Qwen2.5-1.5B" 
#model_name = "Qwen/Qwen2.5-0.5B"          
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
#model = AutoModel.from_pretrained(model_name) 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained( 
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using Device: {device}")

# 如果有GPU，显示显卡信息
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# 把模型移到GPU上！
model = model.to(device)

In [ ]:
# 查看DeepSeek tokenizer的具体信息
print("Tokenizer:", type(tokenizer).__name__)
print("Vocab size:", tokenizer.vocab_size)
print("Special token:")
print(" - CLS:", tokenizer.cls_token)
print(" - SEP:", tokenizer.sep_token) 
print(" - PAD:", tokenizer.pad_token)
print(" - UNK:", tokenizer.unk_token)

In [ ]:
test_texts = "All happy families are alike; each unhappy family is unhappy in its own way."
tokens = tokenizer.tokenize(test_texts)
token_ids = tokenizer.encode(test_texts)
print(f"\nOriginal: {test_texts}")
print(f"Tokens: {tokens}")
print(f"Token IDs: {token_ids[:30]}...")
print(f"toke num: {len(tokens)}")

In [ ]:
def explain_space_handling():
    examples = [
        "hello world",  
        "hello  world", 
        "hello",         
        " hello",     
    ]
    
    for text in examples:
        tokens = tokenizer.tokenize(text)
        print(f"'{text}' → {tokens}")
        for token in tokens:
            if 'Ġ' in token:
                print(f" Note that  '{token}' has a space before it")
                
explain_space_handling()

In [ ]:
embedding_layer = model.get_input_embeddings()
embedding_layer

In [ ]:
import torch
import torch.nn.functional as F

device = next(model.parameters()).device

def get_word_embedding(word, model, tokenizer):
    embedding_layer = model.get_input_embeddings()
    token_ids = tokenizer.encode(word, add_special_tokens=False)

    token_ids = torch.tensor(token_ids, device=device)

    with torch.no_grad():
        embeddings = embedding_layer(token_ids)   # [num_tokens, dim]
        return embeddings.mean(dim=0)              # 平均更稳

def cosine_similarity_torch(vec1, vec2):
    return F.cosine_similarity(
        vec1.unsqueeze(0), 
        vec2.unsqueeze(0), 
        dim=1
    ).item()

# 单词列表
words = ["cat", "dog", "lion", "wind", "rain", "snow", "run", "walk", "jump"]

print("🔍 Similarity matrix")
print("=" * 120)
print(" " * 12 + "".join([f"{word:>10}" for word in words]))

# 计算 embedding
word_embeddings = {
    word: get_word_embedding(word, model, tokenizer)
    for word in words
}

# 相似度矩阵
for word1 in words:
    print(f"{word1:>12}: ", end="")
    for word2 in words:
        sim = cosine_similarity_torch(
            word_embeddings[word1],
            word_embeddings[word2]
        )

        if sim > 0.2:
            print(f"\033[92m{sim:>10.3f}\033[0m", end="")  
        elif sim < 0.1:
            print(f"\033[91m{sim:>10.3f}\033[0m", end="")  
        else:
            print(f"{sim:>10.3f}", end="")
    print()


In [ ]:
def robust_generate(prompt, model, tokenizer, max_new_tokens=50, entropy=0.5):

    device = model.device
    
    print(f"Input: '{prompt}'")
    print(f"Entropy: {entropy} ")
    
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    input_ids = inputs.input_ids
    
    generated_tokens = []
    
    for i in range(max_new_tokens):
        with torch.no_grad():
            outputs = model(input_ids=input_ids)
            

            if hasattr(outputs, 'logits'):
                logits = outputs.logits
            else:
                logits = outputs.last_hidden_state
            
            next_token_logits = logits[:, -1, :]
            

            if entropy == 0.0:
                next_token_id = torch.argmax(next_token_logits, dim=-1, keepdim=True)
                strategy = "Greedy"
                
            else:

                temperature = 0.1 + entropy * 1.9 
                next_token_logits = next_token_logits / temperature
                probs = torch.softmax(next_token_logits, dim=-1)
                next_token_id = torch.multinomial(probs, num_samples=1)
                strategy = f"temperature(t={temperature:.1f})"
            
            new_token = tokenizer.decode(next_token_id[0], skip_special_tokens=True)
            generated_tokens.append(new_token)
            
            print(f"Token {i+1}: '{new_token}' ({strategy})")
            
            input_ids = torch.cat([input_ids, next_token_id], dim=1)
            
            # 停止条件
            if next_token_id.item() == tokenizer.eos_token_id:
                break
            if new_token in ['\n', '.', '!', '?', '。', '！', '？']:
                break
    
    generated_text = prompt + ''.join(generated_tokens)
    print(f"Test: {generated_text}")
    return generated_text

In [ ]:
def ask_question(question, model, tokenizer, max_answer_tokens=50, entropy=0.5):
    
    # 构建提示词，明确要求简短回答
    prompt = f"Question：{question}\n"
    
    print(f"🤔 question: {question}")

    
    # 使用robust_generate生成回答
    full_response = robust_generate(prompt, model, tokenizer, max_new_tokens=max_answer_tokens, entropy=0.5)
    
    # 提取回答部分（去掉问题）
    answer = full_response.replace(prompt, "").strip()
    
    print(f"\n🎯 Final answer: {answer}")
    print(f"📊 Length: {len(answer)} 字")
    
    return answer

In [ ]:
question = "孔子是谁"
answer = ask_question(question, model, tokenizer, max_answer_tokens=50, entropy=0.2)

In [14]:
def show_next_token_probabilities(prompt, model, tokenizer, top_k=20):

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        
        # 获取logits
        if hasattr(outputs, 'logits'):
            logits = outputs.logits
        else:
            logits = outputs.last_hidden_state
        
        # 最后一个token的logits
        next_token_logits = logits[:, -1, :]
        
        # 计算概率
        probs = torch.softmax(next_token_logits, dim=-1)
        
        # 获取前top_k个最可能的token
        top_probs, top_indices = torch.topk(probs, top_k)
        
        print(f"🔍 提示: '{prompt}'")
        print(f"📊 前{top_k}个最可能的下一个token:\n")
        
        for i, (prob, idx) in enumerate(zip(top_probs[0], top_indices[0])):
            token_text = tokenizer.decode([idx])
            # 处理特殊字符的显示
            display_text = repr(token_text)[1:-1]  # 去掉引号
            
            print(f"{i+1:2d}. '{display_text:10s}' (ID: {idx:5d}) - 概率: {prob.item():.4f}")

# 测试当前状态
current_prompt = "问题：孔子是谁\n孔子是我国古代的大思想家，"
show_next_token_probabilities(current_prompt, model, tokenizer, top_k=100)

🔍 提示: '问题：孔子是谁
孔子是我国古代的大思想家，'
📊 前100个最可能的下一个token:

 1. '大         ' (ID: 26288) - 概率: 0.4290
 2. '儒家        ' (ID: 114358) - 概率: 0.1371
 3. '也是        ' (ID: 100000) - 概率: 0.0832
 4. '教育        ' (ID: 99460) - 概率: 0.0435
 5. '他         ' (ID: 42411) - 概率: 0.0363
 6. '是         ' (ID: 20412) - 概率: 0.0262
 7. '他是        ' (ID: 104878) - 概率: 0.0196
 8. '政治        ' (ID: 101091) - 概率: 0.0169
 9. '我国        ' (ID: 101055) - 概率: 0.0147
10. '也         ' (ID: 74763) - 概率: 0.0113
11. '同时也是      ' (ID: 111733) - 概率: 0.0104
12. '春秋        ' (ID: 109385) - 概率: 0.0098
13. '又是        ' (ID: 104458) - 概率: 0.0090
14. '是我国       ' (ID: 108659) - 概率: 0.0088
15. '伟大的       ' (ID: 107792) - 概率: 0.0079
16. '他的        ' (ID: 100648) - 概率: 0.0077
17. '被         ' (ID: 99250) - 概率: 0.0056
18. '著名        ' (ID: 102280) - 概率: 0.0054
19. '儒         ' (ID: 102907) - 概率: 0.0038
20. '被称为       ' (ID: 106253) - 概率: 0.0037
21. '有         ' (ID: 18830) - 概率: 0.0034
22. '哲学        ' (ID: 105384) - 概率: 0.0033
23. '中国  